# ETL Pipeline with Python and Google BigQuery

This notebook demonstrates a complete ETL process (Extract–Transform–Load) using Python, pandas, and Google BigQuery.  
It shows how to authenticate with Google Cloud, process a dataset, load it into a BigQuery table, and validate the result using SQL.

---


In [ ]:
# ============================================================
# 1. INITIAL SETUP
# ============================================================

import os
import pandas as pd
from google.cloud import bigquery
from google.auth import load_credentials_from_file

# Path to credentials
creds_path = os.path.join("credentials", "service_account.json")

if not os.path.isfile(creds_path):
    raise FileNotFoundError(
        f"Credentials file not found at:\n{creds_path}"
    )

# Google Cloud project
project_id = "eloquent-hangar-474417-t1"

# Initialize BigQuery client
creds, _ = load_credentials_from_file(creds_path)
client = bigquery.Client(credentials=creds, project=project_id)

print("BigQuery client successfully initialized.")


In [ ]:
# ============================================================
# 2. EXTRACT — Load dataset from remote repository
# ============================================================

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"

df_raw = pd.read_csv(url)

print("Rows / Columns:", df_raw.shape)
df_raw.head()


In [ ]:
# ============================================================
# 3. TRANSFORM — Clean and aggregate data
# ============================================================

df_transformed = df_raw.groupby("species", as_index=False).mean()

df_transformed


In [ ]:
# ============================================================
# 4. LOAD — Upload transformed data to BigQuery
# ============================================================

dataset_id = "demo_dataset"
table_id = "iris_summary"

table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Load to BigQuery
job = client.load_table_from_dataframe(df_transformed, table_ref)
job.result()

print(f"Table loaded successfully: {table_ref}")


In [ ]:
# ============================================================
# 5. VALIDATION — Query table from BigQuery
# ============================================================

query = f"SELECT * FROM `{table_ref}` LIMIT 10"
df_check = client.query(query).to_dataframe()

df_check


# Conclusion

The ETL pipeline ran successfully:

- **Extract:** loaded a remote public dataset  
- **Transform:** aggregated data using pandas  
- **Load:** uploaded processed data into BigQuery  
- **Validate:** executed SQL query to verify stored data  

This notebook represents a clean, functional example of integrating Python with Google BigQuery for real ETL workflows.
